# 1. Install Dependencies and Setup

In [ ]:
!pip install tensorflow  opencv-python matplotlib

In [ ]:
!pip list

In [ ]:
import tensorflow as tf
import os

In [5]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
tf.config.list_physical_devices('GPU')

[]

# 2. Remove dodgy images

In [ ]:
import cv2
import imghdr

In [9]:
data_dir = '/content/drive/MyDrive/images-h' 

In [10]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [11]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# 3. Load Data

In [12]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('/content/drive/MyDrive/images-h')

In [15]:
data_iterator = data.as_numpy_iterator()

In [16]:
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 4. Scale Data

In [18]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()

# 5. Split Data

In [20]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [21]:
train_size

74

In [22]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# 6. Build Deep Learning Model

In [ ]:
train

In [24]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [25]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [27]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

# 7. Train

In [29]:
logdir='logs'

In [30]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [31]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 41s 508ms/step - accuracy: 0.9400 - loss: 0.1814 - val_accuracy: 0.9568 - val_loss: 0.0789
Epoch 2/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 37s 501ms/step - accuracy: 0.9693 - loss: 0.0663 - val_accuracy: 0.9613 - val_loss: 0.0614
Epoch 3/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 39s 515ms/step - accuracy: 0.9670 - loss: 0.0525 - val_accuracy: 0.9762 - val_loss: 0.0578
Epoch 4/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 44s 591ms/step - accuracy: 0.9802 - loss: 0.0464 - val_accuracy: 0.9732 - val_loss: 0.0543
Epoch 5/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 39s 524ms/step - accuracy: 0.9853 - loss: 0.0554 - val_accuracy: 0.9613 - val_loss: 0.0727
Epoch 6/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 39s 525ms/step - accuracy: 0.9848 - loss: 0.0378 - val_accuracy: 0.9688 - val_loss: 0.0620
Epoch 7/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 40s 530ms/step - accuracy: 0.9919 - loss: 0.0266 - val_accuracy: 0.9658 - val_loss: 0.0793
Epoch 8/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 41s 549ms/step - accuracy: 0.9940 - loss: 0.0137 - val_accu

# 8. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 9. Evaluate

In [34]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [35]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

# 10. Test

In [1]:
import cv2

In [ ]:

from matplotlib import image as mpimg
from matplotlib import pyplot as plt
img = mpimg.imread('/content/drive/MyDrive/images-h/validation-images/validation-images/200300.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [42]:
yhat = model.predict(np.expand_dims(resize/255, 0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step


In [ ]:
yhat

In [45]:
if yhat > 0.5: 
    print(f'Predicted class is ')
    print(yhat*100)
else:
    print(f'Predicted class is Mature')
    print(yhat*100)

Predicted class is Mature


# 11. Save the Model

In [46]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','imageclassifiers.h5'))

In [ ]:
new_model = load_model(os.path.join('models','imageclassifier.h5'))

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))